In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

True

In [3]:
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=os.getenv("GOOGLE_API_KEY"))

In [4]:
class BlogState(TypedDict):
    topic: str
    outline: str
    content: str
    

In [14]:
def generate_outline(state: BlogState) -> BlogState:
    prompt = f"Generate a detailed outline for a blog post about '{state['topic']}'."
    outline = model.invoke([{"role": "user", "content": prompt}]).content
    state['outline'] = outline
    return state

def generate_blog(state: BlogState) -> BlogState:
    prompt = f"Write a blog post based on the following outline:\n{state['outline']}"
    content = model.invoke([{"role": "user", "content": prompt}]).content
    state['content'] = content
    return state

def evaluate_blog(state: BlogState) -> BlogState:
    prompt = f"Evaluate the following blog post for clarity, engagement, and SEO optimization:\n{state['content']}"
    evaluation = model.invoke([{"role": "user", "content": prompt}]).content
    state['evaluation'] = evaluation
    return state

In [15]:
graph = StateGraph(BlogState)

#nodes
graph.add_node('generate_outline', generate_outline)
graph.add_node('generate_blog', generate_blog)
graph.add_node('evaluate_blog', lambda state: state)  # Placeholder for evaluation

#edges
graph.add_edge(START, 'generate_outline')
graph.add_edge('generate_outline', 'generate_blog')
graph.add_edge('generate_blog', END)

#compile
workflow = graph.compile()

In [18]:
#execute the workflow
initial_state: BlogState = {'topic': 'The Future of AI in Healthcare'}
output = workflow.invoke(initial_state)
print("Generated Blog Post:\n")
print(output['content'])

evaluation_state = evaluate_blog(output)
print("\nBlog Post Evaluation:\n")
print(evaluation_state['evaluation'])

Generated Blog Post:

## Beyond the Hype: Unpacking the Transformative Future of AI in Healthcare

Imagine a world where diseases are diagnosed years before symptoms appear, where drug development takes months instead of decades, and personalized treatment plans are as unique as your DNA. This isn't science fiction; it's the promise of Artificial Intelligence in healthcare. AI, encompassing machine learning, deep learning, and natural language processing, is rapidly evolving to analyze vast amounts of medical data, identify patterns, and offer insights far beyond human capacity. While AI is already making inroads in areas like diagnostic imaging and data analysis, its full potential is still largely untapped. This post will explore how AI is poised to fundamentally revolutionize healthcare, but its successful integration hinges on addressing significant technological, ethical, and human challenges, ultimately fostering a powerful human-AI partnership.

### The Present Foundations: Wher

In [19]:
print(output['outline'])

Here's a detailed outline for a blog post about 'The Future of AI in Healthcare', designed to be informative, engaging, and well-structured.

---

## Blog Post Outline: The Future of AI in Healthcare: Unlocking a Healthier Tomorrow

**Target Audience:** General public interested in technology and health, healthcare professionals, policy makers, tech enthusiasts.

**Goal:** To provide a comprehensive yet accessible overview of AI's potential in healthcare, balancing optimism with a realistic look at challenges and ethical considerations.

---

### **I. Catchy Title Options:**

*   **The AI Doctor Will See You Now: Navigating Healthcare's Digital Tomorrow**
*   Healing Algorithms: How AI is Reshaping the Future of Medicine
*   **Beyond the Hype: Unpacking the Transformative Future of AI in Healthcare**
*   AI in Healthcare: Unlocking a Healthier Tomorrow
*   From Diagnosis to Discovery: The Revolutionary Role of AI in Modern Medicine

---

### **II. Introduction (Approx. 150-200 words)**

In [20]:
#generate the evaluation
evaluation_state = evaluate_blog(output)
print("\nBlog Post Evaluation:\n")
print(evaluation_state['evaluation'])


Blog Post Evaluation:

This blog post is exceptionally well-written and covers a complex topic with impressive clarity and depth. Here's a detailed evaluation:

---

### Overall Impression

This is a comprehensive, well-structured, and highly informative blog post. It strikes an excellent balance between highlighting the exciting potential of AI in healthcare and responsibly addressing the critical challenges. The language is accessible yet authoritative, making it suitable for a broad audience interested in the future of healthcare.

---

### 1. Clarity

**Strengths:**

*   **Excellent Structure:** The post uses clear, descriptive headings (H2s and H3s) that logically guide the reader through the topic. This makes it easy to follow the progression from current applications to future potential, challenges, and the ultimate vision.
*   **Accessible Language:** While discussing complex technological and medical concepts, the language remains clear and avoids excessive jargon. Where tech